In [1]:
# !git clone --recursive https://github.com/microsoft/LightGBM
# %cd /content/LightGBM
# !mkdir build
# !cmake -DUSE_GPU=1 #avoid ..
# !make -j$(nproc)
# !sudo apt-get -y install python-pip
# #!sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
# %cd /content/LightGBM/python-package
# !sudo python setup.py install — precompile
# %cd /content/

In [2]:
!gdown --id 196BS6Wq4gcL-JXWPFOryBidS9KC9CtJp
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c instacart-market-basket-analysis

Downloading...
From: https://drive.google.com/uc?id=196BS6Wq4gcL-JXWPFOryBidS9KC9CtJp
To: /content/kaggle.json
100% 65.0/65.0 [00:00<00:00, 30.6kB/s]
order_products__prior.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
aisles.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
order_products__train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
orders.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
departments.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
products.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
import gc
import pandas as pd
import numpy as np
import os
import lightgbm as lgb
import json
import sklearn.metrics
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from scipy.sparse import dok_matrix, coo_matrix
from sklearn.utils.multiclass import  type_of_target

In [4]:
# !unzip orders.csv.zip
# !unzip order_products__prior.csv.zip
# !unzip order_products__train.csv.zip
# !unzip aisles.csv.zip
# !unzip departments.csv.zip
# !unzip products.csv.zip

In [5]:
# !mkdir data
# !cp drive/My\ Drive/eCloudvalley/chunk_0.pkl data/
# !cp drive/My\ Drive/eCloudvalley/orders_comsum.pkl data/
# !cp drive/My\ Drive/eCloudvalley/previous_products.pkl data/
# !cp drive/My\ Drive/eCloudvalley/product_period.pkl data/
# !cp drive/My\ Drive/eCloudvalley/product_periods_stat.pkl data/ 
# !cp drive/My\ Drive/eCloudvalley/user_aisle_products.pkl data/
# !cp drive/My\ Drive/eCloudvalley/user_department_products.pkl data/
# !cp drive/My\ Drive/eCloudvalley/prod2vec.pkl data/
# !cp drive/My\ Drive/eCloudvalley/product_embeddings.pkl data/

In [6]:
def fscore(true_value_matrix, prediction, order_index, product_index, rows, cols, threshold=[0.5]):

    prediction_value_matrix = coo_matrix((prediction, (order_index, product_index)), shape=(rows, cols), dtype=np.float32)
    # prediction_value_matrix.eliminate_zeros()

    return list(map(lambda x: f1_score(true_value_matrix, prediction_value_matrix > x, average='samples'), threshold))

In [7]:
path = "/content"

aisles = pd.read_csv(os.path.join(path, "aisles.csv"), dtype={'aisle_id': np.uint8, 'aisle': 'category'})
departments = pd.read_csv(os.path.join(path, "departments.csv"),
                          dtype={'department_id': np.uint8, 'department': 'category'})
order_prior = pd.read_csv(os.path.join(path, "order_products__prior.csv"), dtype={'order_id': np.uint32,
                                           'product_id': np.uint16,
                                           'add_to_cart_order': np.uint8,
                                           'reordered': bool})
order_train = pd.read_csv(os.path.join(path, "order_products__train.csv"), dtype={'order_id': np.uint32,
                                           'product_id': np.uint16,
                                           'add_to_cart_order': np.uint8,
                                           'reordered': bool})
orders = pd.read_csv(os.path.join(path, "orders.csv"), dtype={'order_id': np.uint32,
                                 'user_id': np.uint32,
                                 'eval_set': 'category',
                                 'order_number': np.uint8,
                                 'order_dow': np.uint8,
                                 'order_hour_of_day': np.uint8})

products = pd.read_csv(os.path.join(path, "products.csv"), dtype={'product_id': np.uint16,
                                   'aisle_id': np.uint8,
                                    'department_id': np.uint8})

product_embeddings = pd.read_pickle('data/product_embeddings.pkl')
embedings = list(str(x) for x in range(32))
product_embeddings = product_embeddings[embedings + ['product_id']]

order_train = pd.read_pickle(os.path.join(path, 'data/chunk_0.pkl'))
order_test = order_train.loc[order_train.eval_set == "test", ['order_id', 'product_id']]
order_train = order_train.loc[order_train.eval_set == "train", ['order_id',  'product_id',  'reordered']]

product_periods = pd.read_pickle(os.path.join(path, 'data/product_periods_stat.pkl')).fillna(9999)

print(order_train.columns)

Index(['order_id', 'product_id', 'reordered'], dtype='object')


In [8]:
#order_products_all = pd.concat([order_prior, order_train])
order_priors = orders[orders.eval_set == 'prior']

user_products = pd.merge(order_prior, order_priors, on='order_id')[['order_id', 'product_id', 'user_id', 'order_number']]
user_products['order_number_rev'] = user_products.groupby('user_id').order_number.transform(np.max) - user_products.order_number

#total buy = how many product_id user_id has bought
total_buy = user_products.groupby(['user_id', 'product_id']).size().reset_index()
total_buy.columns = ['user_id', 'product_id','total_buy']

user_products = pd.merge(user_products, total_buy, on=['user_id',	'product_id'])
user_products['total_buy_ratio'] = user_products.total_buy / user_products.groupby('user_id').order_number.transform(np.max)
total_buy_n5 = user_products[user_products.order_number_rev < 5].groupby(['user_id', 'product_id']).size().reset_index()
total_buy_n5.columns = ['user_id', 'product_id','total_buy_n5']

def min_5(x):
  return min(np.max(x), 5)

user_products = pd.merge(user_products, total_buy_n5, on=['user_id',	'product_id'])
user_products['total_buy_n5_ratio'] = user_products.total_buy_n5 / user_products.groupby('user_id').order_number.transform(min_5)
user_products = user_products[['user_id',	'product_id', 'total_buy_ratio',	'total_buy_n5',	'total_buy_n5_ratio']]
user_products = user_products.drop_duplicates()
user_products

,user_id,product_id,total_buy_ratio,total_buy_n5,total_buy_n5_ratio
0,202279,33120,0.625000,3,0.6
5,202279,28985,0.625000,2,0.4
10,202279,45918,0.625000,3,0.6
15,202279,30035,0.375000,2,0.4
18,202279,17794,0.875000,4,0.8
...,...,...,...,...,...
19805095,25247,45309,0.037037,1,0.2
19805096,25247,21162,0.037037,1,0.2
19805097,25247,35211,0.037037,1,0.2
19805098,25247,11352,0.037037,1,0.2


In [9]:
prob = pd.merge(order_prior, orders, on='order_id')
print(prob.columns)
prob = prob.groupby(['product_id', 'user_id'])\
    .agg({'reordered':'sum', 'user_id': 'size'})
print(prob.columns)

prob.rename(columns={'sum': 'reordered', 'user_id': 'total'}, inplace=True)

prob.reordered = (prob.reordered > 0).astype(np.float32)
prob.total = (prob.total > 0).astype(np.float32)
prob['reorder_prob'] = prob.reordered / prob.total
prob = prob.groupby('product_id').agg({'reorder_prob': 'mean'}).rename(columns={'mean': 'reorder_prob'}).reset_index()


prod_stat = order_prior.groupby('product_id').agg({'reordered': ['sum', 'size'], 'add_to_cart_order':'mean'})
prod_stat.columns = prod_stat.columns.levels[1]
prod_stat.rename(columns={'sum':'prod_add_to_card_mean','size':'prod_orders', 'mean': 'prod_reorders'}, inplace=True)
prod_stat.reset_index(inplace=True)

prod_stat['reorder_ration'] = prod_stat['prod_reorders'] / prod_stat['prod_orders']

prod_stat = pd.merge(prod_stat, prob, on='product_id')

Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered', 'user_id',
       'eval_set', 'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order'],
      dtype='object')
Index(['reordered', 'user_id'], dtype='object')


In [10]:
user_stat = orders.loc[orders.eval_set == 'prior', :].groupby('user_id').agg({'order_number': 'max',
                                         'days_since_prior_order': ['sum','mean','median']})
user_stat.columns = user_stat.columns.droplevel(0)
user_stat.rename(columns={'max': 'user_orders',
             'sum': 'user_order_starts_at',
             'mean': 'user_mean_days_since_prior',
             'median': 'user_median_days_since_prior'}, inplace=True)
user_stat.reset_index(inplace=True)

orders_products = pd.merge(orders, order_prior, on="order_id")

user_order_stat = orders_products.groupby('user_id').agg({'user_id': 'size',
                              'reordered': 'sum',
                              "product_id": lambda x: x.nunique()})

user_order_stat.rename(columns={'user_id': 'user_total_products',
                                'product_id': 'user_distinct_products',
                                'reordered': 'user_reorder_ratio'}, inplace=True)

user_order_stat.reset_index(inplace=True)
user_order_stat.user_reorder_ratio = user_order_stat.user_reorder_ratio / user_order_stat.user_total_products

user_stat = pd.merge(user_stat, user_order_stat, on='user_id')
user_stat['user_average_basket'] = user_stat.user_total_products / user_stat.user_orders

products

In [11]:
prod_usr = orders_products.groupby(['product_id']).agg({'user_id': lambda x: x.nunique()})
prod_usr.rename(columns={'user_id':'prod_users_unq'}, inplace=True)
prod_usr.reset_index(inplace=True)

prod_usr_reordered = orders_products.loc[orders_products.reordered, :].groupby(['product_id']).agg({'user_id': lambda x: x.nunique()})
prod_usr_reordered.rename(columns={'user_id': 'prod_users_unq_reordered'}, inplace=True)
prod_usr_reordered.reset_index(inplace=True)

order_stat = orders_products.groupby('order_id').agg({'order_id': 'size'}).rename(columns={'order_id': 'order_size'}).reset_index()

orders_products = pd.merge(orders_products, order_stat, on='order_id')
orders_products['add_to_cart_order_inverted'] = orders_products.order_size - orders_products.add_to_cart_order
orders_products['add_to_cart_order_relative'] = orders_products.add_to_cart_order / orders_products.order_size

In [12]:
data = orders_products.groupby(['user_id', 'product_id']).agg({'user_id': 'size',
                                 'order_number': ['min', 'max'],
                                 'add_to_cart_order': ['mean', 'median'],
                                 'days_since_prior_order': ['mean', 'median'],
                                 'order_dow': ['mean', 'median'],
                                 'order_hour_of_day': ['mean', 'median'],
                                 'add_to_cart_order_inverted': ['mean', 'median'],
                                 'add_to_cart_order_relative': ['mean', 'median'],
                                 'reordered': ['sum']})

data.columns = data.columns.droplevel(0)
data.columns = ['up_orders', 'up_first_order', 'up_last_order', 'up_mean_cart_position', 'up_median_cart_position',
                'days_since_prior_order_mean', 'days_since_prior_order_median', 'order_dow_mean',
                'order_dow_median',
                'order_hour_of_day_mean', 'order_hour_of_day_median',
                'add_to_cart_order_inverted_mean', 'add_to_cart_order_inverted_median',
                'add_to_cart_order_relative_mean', 'add_to_cart_order_relative_median',
                'reordered_sum'
                ]

data['user_product_reordered_ratio'] = (data.reordered_sum + 1.0) / data.up_orders

data.reset_index(inplace=True)

data = pd.merge(data, prod_stat, on='product_id')
data = pd.merge(data, user_stat, on='user_id')

data['up_order_rate'] = data.up_orders / data.user_orders
data['up_orders_since_last_order'] = data.user_orders - data.up_last_order
data['up_order_rate_since_first_order'] = data.user_orders / (data.user_orders - data.up_first_order + 1)
data.shape

(13307953, 35)

In [13]:
data = pd.merge(data, user_products, on=['user_id', 'product_id'], how='left')
data.shape

(13307953, 38)

In [14]:
user_dep_stat = pd.read_pickle('data/user_department_products.pkl')
user_aisle_stat = pd.read_pickle('data/user_aisle_products.pkl')

Train Data

In [15]:
print(order_train.shape)
order_train = pd.merge(order_train, products, on='product_id')
print(order_train.shape)
order_train = pd.merge(order_train, orders, on='order_id')
print(order_train.shape)
order_train = pd.merge(order_train, user_dep_stat, on=['user_id', 'department_id'])
print(order_train.shape)
order_train = pd.merge(order_train, user_aisle_stat, on=['user_id', 'aisle_id'])
print(order_train.shape)

order_train = pd.merge(order_train, prod_usr, on='product_id')
print(order_train.shape)
order_train = pd.merge(order_train, prod_usr_reordered, on='product_id', how='left')
order_train.prod_users_unq_reordered.fillna(0, inplace=True)
print(order_train.shape)

order_train = pd.merge(order_train, data, on=['product_id', 'user_id'])
print(order_train.shape)

order_train['aisle_reordered_ratio'] = order_train.aisle_reordered / order_train.user_orders
order_train['dep_reordered_ratio'] = order_train.dep_reordered / order_train.user_orders

order_train = pd.merge(order_train, product_periods, on=['user_id',  'product_id'])

(8474661, 3)
(8474661, 6)
(8474661, 12)
(8474661, 14)
(8474661, 16)
(8474661, 17)
(8474661, 18)
(8474661, 54)


Testing Data

In [16]:
order_test = pd.merge(order_test, products, on='product_id')
order_test = pd.merge(order_test, orders, on='order_id')
order_test = pd.merge(order_test, user_dep_stat, on=['user_id', 'department_id'])
order_test = pd.merge(order_test, user_aisle_stat, on=['user_id', 'aisle_id'])

order_test = pd.merge(order_test, prod_usr, on='product_id')
order_test = pd.merge(order_test, prod_usr_reordered, on='product_id', how='left')
order_train.prod_users_unq_reordered.fillna(0, inplace=True)

order_test = pd.merge(order_test, data, on=['product_id', 'user_id'])

order_test['aisle_reordered_ratio'] = order_test.aisle_reordered / order_test.user_orders
order_test['dep_reordered_ratio'] = order_test.dep_reordered / order_test.user_orders

order_test = pd.merge(order_test, product_periods, on=['user_id', 'product_id'])

order_train = pd.merge(order_train, product_embeddings, on=['product_id'])
order_test = pd.merge(order_test, product_embeddings, on=['product_id'])

print('data is joined')

data is joined


In [17]:
features = [
    # 'reordered_dow_ration', 'reordered_dow', 'reordered_dow_size',
    # 'reordered_prev', 'add_to_cart_order_prev', 'order_dow_prev', 'order_hour_of_day_prev',
    'user_product_reordered_ratio', 'reordered_sum',
    'add_to_cart_order_inverted_mean', 'add_to_cart_order_relative_mean',
    'reorder_prob',
    'last', 'prev1', 'prev2', 'median', 'mean',
    'dep_reordered_ratio', 'aisle_reordered_ratio',
    'aisle_products',
    'aisle_reordered',
    'dep_products',
    'dep_reordered',
    'prod_users_unq', 'prod_users_unq_reordered',
    'order_number', 'prod_add_to_card_mean',
    'days_since_prior_order',
    'order_dow', 'order_hour_of_day',
    'reorder_ration',
    'user_orders', 'user_order_starts_at', 'user_mean_days_since_prior',
    # 'user_median_days_since_prior',
    'user_average_basket', 'user_distinct_products', 'user_reorder_ratio', 'user_total_products',
    'prod_orders', 'prod_reorders',
    'up_order_rate', 'up_orders_since_last_order', 'up_order_rate_since_first_order',
    'up_orders', 'up_first_order', 'up_last_order', 'up_mean_cart_position',
    # 'up_median_cart_position',
    'days_since_prior_order_mean',
    # 'days_since_prior_order_median',
    'order_dow_mean',
    # 'order_dow_median',
    'order_hour_of_day_mean', 'total_buy_ratio',	'total_buy_n5',	'total_buy_n5_ratio'
    # 'order_hour_of_day_median'
]
features.extend(embedings)
categories = ['product_id', 'aisle_id', 'department_id', 'order_id', 'user_id']
#features.extend(embedings)
cat_features = ','.join(map(lambda x: str(x + len(features)), range(len(categories))))
features.extend(categories)

print('not included', set(order_train.columns.tolist()) - set(features))

data = order_train[features]
labels = order_train[['reordered']].values.astype(np.float32).flatten()

data_val = order_test[features]

assert data.shape[0] == 8474661

not included {'days_since_prior_order_median', 'product_name', 'add_to_cart_order_inverted_median', 'reordered', 'user_median_days_since_prior', 'order_dow_median', 'up_median_cart_position', 'add_to_cart_order_relative_median', 'order_hour_of_day_median', 'eval_set'}


In [18]:
ind = []
for i in range(len(data.dtypes)):
  if data.dtypes[i] == 'float64':
    ind.append(i)
data.iloc[:, ind] = data.iloc[:, ind].astype('float32')

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [19]:
gc.collect()
data.to_pickle('data_all.pkl')

In [20]:
!cp data_all.pkl drive/My\ Drive/eCloudvalley

In [ ]:
data.to_pickle('data_all.pkl')
!cp data_all.pkl drive/My\ Drive/eCloudvalley

In [19]:
data_val.to_pickle('data_val.pkl')

In [20]:
!cp data_val.pkl drive/My\ Drive/eCloudvalley

In [21]:
del data_val
gc.collect()

11

In [24]:
pd.Series(labels).to_pickle('labels_all.pkl')
!cp labels_all.pkl drive/My\ Drive/eCloudvalley

In [ ]:
labels1 = pd.Series(labels[:2000000])
labels2 = pd.Series(labels[2000000:4000000])
labels3 = pd.Series(labels[4000000:6000000])
labels4 = pd.Series(labels[6000000:8000000])
labels5 = pd.Series(labels[8000000:])

In [ ]:
labels1.to_pickle('labels1.pkl')
labels2.to_pickle('labels2.pkl')
labels3.to_pickle('labels3.pkl')
labels4.to_pickle('labels4.pkl')
labels5.to_pickle('labels5.pkl')

In [ ]:
!cp labels1.pkl drive/My\ Drive/eCloudvalley
!cp labels2.pkl drive/My\ Drive/eCloudvalley
!cp labels3.pkl drive/My\ Drive/eCloudvalley
!cp labels4.pkl drive/My\ Drive/eCloudvalley
!cp labels5.pkl drive/My\ Drive/eCloudvalley

In [18]:
data1 = data.iloc[:2000000]
data1.to_pickle('data1.pkl')
!cp data1.pkl drive/My\ Drive/eCloudvalley
del data1

data2 = data.iloc[200000:4000000]
data2.to_pickle('data2.pkl')
!cp data2.pkl drive/My\ Drive/eCloudvalley
del data2

data3 = data.iloc[4000000:6000000]
data3.to_pickle('data3.pkl')
!cp data3.pkl drive/My\ Drive/eCloudvalley
del data3

data4 = data.iloc[6000000:8000000]
data4.to_pickle('data4.pkl')
!cp data4.pkl drive/My\ Drive/eCloudvalley
del data4

data5 = data.iloc[8000000:]
data5.to_pickle('data5.pkl')
!cp data5.pkl drive/My\ Drive/eCloudvalley
del data5



In [ ]:
gc.collect()

0

In [ ]:

# !cp data2.pkl drive/My\ Drive/eCloudvalley
# !cp data3.pkl drive/My\ Drive/eCloudvalley
# !cp data4.pkl drive/My\ Drive/eCloudvalley
# !cp data5.pkl drive/My\ Drive/eCloudvalley

In [ ]:
lgb_train = lgb.Dataset(data1, labels1, categorical_feature=['product_id', 'aisle_id', 'department_id'])

# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss', 'auc'},
    'num_leaves': 256,
    'min_sum_hessian_in_leaf': 20,
    'max_depth': 12,
    'learning_rate': 0.05,
    'feature_fraction': 0.6,
    # 'bagging_fraction': 0.9,
    # 'bagging_freq': 3,
    'verbose': 1
}

print('Start training...')

NameError: ignored

In [ ]:
data2

,user_product_reordered_ratio,reordered_sum,add_to_cart_order_inverted_mean,add_to_cart_order_relative_mean,reorder_prob,last,prev1,prev2,median,mean,dep_reordered_ratio,aisle_reordered_ratio,aisle_products,aisle_reordered,dep_products,dep_reordered,prod_users_unq,prod_users_unq_reordered,order_number,prod_add_to_card_mean,days_since_prior_order,order_dow,order_hour_of_day,reorder_ration,user_orders,user_order_starts_at,user_mean_days_since_prior,user_average_basket,user_distinct_products,user_reorder_ratio,user_total_products,prod_orders,prod_reorders,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order,up_orders,up_first_order,up_last_order,up_mean_cart_position,...,27,28,29,30,31,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,product_id,aisle_id,department_id
200001,1.0,0.0,2.000000,0.600000,0.664226,14.0,9999.0,9999.0,9999.0,9999.00,0.250000,0.000000,3,0.0,7,1.0,55037,36557.0,5,186884.0,14.0,0,15,0.000031,4,49.0,16.333333,6.000000,21,0.125000,24,241921,7.429640,0.250000,0,4.000000,1,4,4,3.000000,...,0.095473,0.101987,-0.073177,0.164218,0.339498,-0.059479,0.194073,-0.159856,-0.353550,0.121405,0.058438,-0.211632,-0.165866,0.664879,-0.083461,-0.483873,-0.101163,0.036450,-0.489091,0.076628,-0.158616,-0.041642,0.940578,-0.621478,0.750606,-0.142164,-0.006857,0.276950,-0.307169,0.008369,0.051761,0.101532,0.095473,0.101987,-0.073177,0.164218,0.339498,21903,123,4
200002,1.0,4.0,3.600000,0.333030,0.664226,7.0,14.0,8.0,11.0,10.75,2.166667,0.833333,2,5.0,7,13.0,55037,36557.0,7,186884.0,7.0,1,22,0.000031,6,43.0,8.600000,4.333333,12,0.538462,26,241921,7.429640,0.833333,0,1.000000,5,1,6,1.400000,...,0.095473,0.101987,-0.073177,0.164218,0.339498,-0.059479,0.194073,-0.159856,-0.353550,0.121405,0.058438,-0.211632,-0.165866,0.664879,-0.083461,-0.483873,-0.101163,0.036450,-0.489091,0.076628,-0.158616,-0.041642,0.940578,-0.621478,0.750606,-0.142164,-0.006857,0.276950,-0.307169,0.008369,0.051761,0.101532,0.095473,0.101987,-0.073177,0.164218,0.339498,21903,123,4
200003,1.0,0.0,8.000000,0.272727,0.664226,114.0,9999.0,9999.0,9999.0,9999.00,1.636364,0.000000,1,0.0,12,36.0,55037,36557.0,23,186884.0,7.0,1,22,0.000031,22,260.0,12.380952,6.909091,31,0.796053,152,241921,7.429640,0.045455,9,2.200000,1,13,13,3.000000,...,0.095473,0.101987,-0.073177,0.164218,0.339498,-0.059479,0.194073,-0.159856,-0.353550,0.121405,0.058438,-0.211632,-0.165866,0.664879,-0.083461,-0.483873,-0.101163,0.036450,-0.489091,0.076628,-0.158616,-0.041642,0.940578,-0.621478,0.750606,-0.142164,-0.006857,0.276950,-0.307169,0.008369,0.051761,0.101532,0.095473,0.101987,-0.073177,0.164218,0.339498,21903,123,4
200004,1.0,1.0,2.000000,0.458333,0.664226,69.0,29.0,9999.0,29.0,29.00,0.250000,0.250000,1,1.0,2,1.0,55037,36557.0,5,186884.0,19.0,0,10,0.000031,4,79.0,26.333333,5.750000,16,0.304348,23,241921,7.429640,0.500000,2,1.000000,2,1,2,1.500000,...,0.095473,0.101987,-0.073177,0.164218,0.339498,-0.059479,0.194073,-0.159856,-0.353550,0.121405,0.058438,-0.211632,-0.165866,0.664879,-0.083461,-0.483873,-0.101163,0.036450,-0.489091,0.076628,-0.158616,-0.041642,0.940578,-0.621478,0.750606,-0.142164,-0.006857,0.276950,-0.307169,0.008369,0.051761,0.101532,0.095473,0.101987,-0.073177,0.164218,0.339498,21903,123,4
200005,1.0,0.0,8.000000,0.384615,0.664226,31.0,9999.0,9999.0,9999.0,9999.00,0.600000,0.000000,1,0.0,10,3.0,55037,36557.0,6,186884.0,11.0,0,17,0.000031,5,41.0,10.250000,13.400000,42,0.373134,67,241921,7.429640,0.200000,1,2.500000,1,4,4,5.000000,...,0.095473,0.101987,-0.073177,0.164218,0.339498,-0.059479,0.194073,-0.159856,-0.353550,0.121405,0.058438,-0.211632,-0.165866,0.664879,-0.083461,-0.483873,-0.101163,0.036450,-0.489091,0.076628,-0.158616,-0.041642,0.940578,-0.621478,0.750606,-0.142164,-0.006857,0.276950,-0.307169,0.008369,0.051761,0.101532,0.095473,0.101987,-0.073177,0.164218,0.339498,21903,123,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
# train
gbm = lgb.train(params, lgb_train, num_boost_round=380)

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


In [ ]:


prediction = gbm.predict(data_val)
# prediction = model.predict(data_val)
orders = order_test.order_id.values
products = order_test.product_id.values

result = pd.DataFrame({'product_id': products, 'order_id': orders, 'prediction': prediction})
result.to_pickle('data/prediction_lgbm.pkl')

In [ ]:
orders = order_test.order_id.values
products = order_test.product_id.values

array([  65432,  203668, 1734476, ..., 3323261,  639533, 2184320],
      dtype=uint64)

In [ ]:
!cp data_val2.pkl drive/My\ Drive/eCloudvalley

In [ ]:
!cp data_val1.pkl drive/My\ Drive/eCloudvalley
!cp data_val2.pkl drive/My\ Drive/eCloudvalley

In [ ]:
data_val.iloc[4000000:].to_pickle('data_val3.pkl')

In [ ]:
data_val1 = data_val.iloc[:25000000]
data_val2 = data_val.iloc[2500000:]

In [ ]:
data_val1.to_pickle('data_val1.pkl')
data_val2.to_pickle('data_val2.pkl')

In [ ]:
data_val

,user_product_reordered_ratio,reordered_sum,add_to_cart_order_inverted_mean,add_to_cart_order_relative_mean,reorder_prob,last,prev1,prev2,median,mean,dep_reordered_ratio,aisle_reordered_ratio,aisle_products,aisle_reordered,dep_products,dep_reordered,prod_users_unq,prod_users_unq_reordered,order_number,prod_add_to_card_mean,days_since_prior_order,order_dow,order_hour_of_day,reorder_ration,user_orders,user_order_starts_at,user_mean_days_since_prior,user_average_basket,user_distinct_products,user_reorder_ratio,user_total_products,prod_orders,prod_reorders,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order,up_orders,up_first_order,up_last_order,up_mean_cart_position,days_since_prior_order_mean,order_dow_mean,order_hour_of_day_mean,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,product_id,aisle_id,department_id
0,1.0,0.0,12.000000,0.076923,0.378882,82.0,9999.0,9999.0,9999.0,9999.0000,0.000000,0.000000,1,0.0,2,0.0,161,61.0,11,4.066116,15.0,4,12,0.667355,10,68.0,7.555556,6.700000,39,0.417910,67,484,323.0,0.100000,8,1.111111,1,2,2,1.000000,1.000000,6.000000,17.000000,-0.534005,0.923381,0.254286,-0.359644,0.949903,0.152821,0.428330,0.142521,0.696405,-0.163492,0.063763,0.673153,-0.431812,-0.327063,0.023732,-0.128016,0.540590,-0.140138,0.283237,-0.566959,-0.414986,-0.618799,-0.682262,1.026910,-0.693585,-0.717572,0.136753,0.976410,0.186198,0.633050,0.234474,-0.022536,17330,27,5
1,1.0,0.0,2.000000,0.846154,0.378882,21.0,9999.0,9999.0,9999.0,9999.0000,1.793651,0.857143,28,54.0,68,113.0,161,61.0,64,4.066116,1.0,5,9,0.667355,63,364.0,5.870968,4.634921,159,0.455479,292,484,323.0,0.015873,5,10.500000,1,58,58,11.000000,7.000000,5.000000,9.000000,-0.534005,0.923381,0.254286,-0.359644,0.949903,0.152821,0.428330,0.142521,0.696405,-0.163492,0.063763,0.673153,-0.431812,-0.327063,0.023732,-0.128016,0.540590,-0.140138,0.283237,-0.566959,-0.414986,-0.618799,-0.682262,1.026910,-0.693585,-0.717572,0.136753,0.976410,0.186198,0.633050,0.234474,-0.022536,17330,27,5
2,1.0,16.0,2.235294,0.649977,0.378882,88.0,4.0,11.0,9.5,13.1875,0.978022,0.758242,8,69.0,20,89.0,161,61.0,92,4.066116,4.0,5,16,0.667355,91,336.0,3.733333,5.164835,71,0.848936,470,484,323.0,0.186813,12,1.096386,17,9,79,3.823529,2.764706,3.117647,13.823529,-0.534005,0.923381,0.254286,-0.359644,0.949903,0.152821,0.428330,0.142521,0.696405,-0.163492,0.063763,0.673153,-0.431812,-0.327063,0.023732,-0.128016,0.540590,-0.140138,0.283237,-0.566959,-0.414986,-0.618799,-0.682262,1.026910,-0.693585,-0.717572,0.136753,0.976410,0.186198,0.633050,0.234474,-0.022536,17330,27,5
3,1.0,0.0,9.000000,0.307692,0.378882,48.0,9999.0,9999.0,9999.0,9999.0000,0.891304,0.586957,17,27.0,32,41.0,161,61.0,47,4.066116,29.0,3,16,0.667355,46,324.0,7.200000,3.804348,98,0.440000,175,484,323.0,0.021739,1,23.000000,1,45,45,4.000000,1.000000,4.000000,15.000000,-0.534005,0.923381,0.254286,-0.359644,0.949903,0.152821,0.428330,0.142521,0.696405,-0.163492,0.063763,0.673153,-0.431812,-0.327063,0.023732,-0.128016,0.540590,-0.140138,0.283237,-0.566959,-0.414986,-0.618799,-0.682262,1.026910,-0.693585,-0.717572,0.136753,0.976410,0.186198,0.633050,0.234474,-0.022536,17330,27,5
4,1.0,1.0,14.500000,0.196594,0.378882,109.0,83.0,9999.0,83.0,83.0000,1.375000,0.750000,14,30.0,49,55.0,161,61.0,41,4.066116,6.0,0,11,0.667355,40,358.0,9.179487,27.475000,180,0.836215,1099,484,323.0,0.050000,10,2.222222,2,23,30,3.500000,9.000000,3.500000,12.000000,-0.534005,0.923381,0.254286,-0.359644,0.949903,0.152821,0.428330,0.142521,0.696405,-0.163492,0.063763,0.673153,-0.431812,-0.327063,0.023732,-0.128016,0.540590,-0.140138,0.283237,-0.566959,-0.414986,-0.618799,-0.682262,1.026910,-0.693585,-0.717572,0.136753,0.976410,0.186198,0.633050,0.234474,-0.022536,17330,27,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
order_test